In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
#초기에는 난수로 hidden_layer와 output_layter의 weights를 결정합니다. 
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [2]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]  # 순전파 진행 
    return activation
#activation = weights의 값과 input값의 곱

def sigmoid(activation):
    return 1.0 / (1.0 + np.exp(-activation)) # 시그모이드 구현
#activation값에 sigmoid를 적용합니다. 

def forward_propagate(network, row): #forward_propagatin 진행
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)  
            neuron['output'] = sigmoid(activation) 
            new_inputs.append(neuron['output']) # 기존의 output을 new input으로 적용합니다.
        inputs = new_inputs
    return inputs

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [3]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output) #순전파 학습과정에 대해서 test를 진행합니다. 

[0.6629970129852887, 0.7253160725279748]


In [4]:
def sigmoid_derivative(output):
    return output * (1.0 - output) # 시그모이드 미분을 위한 함수를 만들어 줍니다. 

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta']) #error값은 weights와 delta값의 곱입니다.
                errors.append(error) 
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) # 역전파시 오차는 예측값-출력값을 뺀 값을 시그모이드 적용
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output']) # 시그모이드 함수를 사용한 역전파 

In [5]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [6]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta'] # 퍼셉트론 학습 규칙
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # 예측값의 오차 합
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [7]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [8]:
n_inputs = len(dataset[0]) - 1 #
#입력층 입력노드의 수 = 입력 변수의 수 
n_outputs = len(set([row[-1] for row in dataset])) 
#범주형 출력노드의 수 = 출력변수의 범주 수
#연속형 출력노드 수 = 출력변수의 갯수
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.9, 20, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.
# l_rate = 학습률, n_epoch = 각계층의 weight를 모두 거치는 횟수
# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로,hint : for문))
for layer in network:
    print(layer)

>epoch=0, lrate=0.900, error=6.049
>epoch=1, lrate=0.900, error=5.318
>epoch=2, lrate=0.900, error=4.914
>epoch=3, lrate=0.900, error=4.260
>epoch=4, lrate=0.900, error=3.682
>epoch=5, lrate=0.900, error=3.112
>epoch=6, lrate=0.900, error=2.622
>epoch=7, lrate=0.900, error=2.216
>epoch=8, lrate=0.900, error=1.885
>epoch=9, lrate=0.900, error=1.614
>epoch=10, lrate=0.900, error=1.389
>epoch=11, lrate=0.900, error=1.202
>epoch=12, lrate=0.900, error=1.048
>epoch=13, lrate=0.900, error=0.921
>epoch=14, lrate=0.900, error=0.816
>epoch=15, lrate=0.900, error=0.728
>epoch=16, lrate=0.900, error=0.654
>epoch=17, lrate=0.900, error=0.592
>epoch=18, lrate=0.900, error=0.539
>epoch=19, lrate=0.900, error=0.493
[{'weights': [-1.7204694744146227, 2.209369600776161, 1.2288173645489384], 'output': 0.01694694058874678, 'delta': -0.002483315799124754}, {'weights': [-0.8059759447531837, 0.7873519761195067, 0.513859122772257], 'output': 0.05988674588950716, 'delta': -0.002385229823253038}]
[{'weights': 

In [9]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # error가 충분히 줄어들때 까지 반복하여 진행

In [10]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))
    # 이해 RE!
    # -> 실제 코드에서는 print('Exepcted=%d, Got=%d' % (row[-1], prediction))

실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
